In [ ]:
%matplotlib inline

4D interpolation
=============

Interpolation of a four-dimensional regular grid.

Quadrivariate
----------------

The
[quadrivariate](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.quadrivariate.html#pyinterp.quadrivariate)
interpolation allows obtaining values at arbitrary points in a 4D space of a
function defined on a grid.

The distribution contains a 4D field `pres_temp_4D.nc` that will be used
in this help. This file is located in the `src/pyinterp/tests/dataset` directory
at the root of the project.

This method performs a bilinear interpolation in 2D space by considering the
axes of longitude and latitude of the grid, then performs a linear interpolation
in the third and fourth dimensions. Its interface is similar to the
[trivariate](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.trivariate.html#pyinterp.trivariate)
class except for a fourth axis, which is handled by this object.

In [ ]:
import cartopy.crs
import matplotlib
import matplotlib.pyplot
import numpy
import pyinterp
import pyinterp.backends.xarray
import pyinterp.tests
import xarray

The first step is to load the data into memory and create the
interpolator object:

In [ ]:
ds = xarray.open_dataset(pyinterp.tests.grid4d_path())
interpolator = pyinterp.backends.xarray.Grid4D(ds.pressure)

We will build a new grid that will be used to build a new interpolated
grid.

In [ ]:
mx, my, mz, mu = numpy.meshgrid(numpy.arange(-125, -70, 0.5),
                                numpy.arange(25, 50, 0.5),
                                numpy.datetime64("2000-01-01T12:00"),
                                0.5,
                                indexing="ij")

We interpolate our grid using a
[classical](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.backends.xarray.Grid4D.quadrivariate.html#pyinterp.backends.xarray.Grid4D.quadrivariate):

In [ ]:
quadrivariate = interpolator.quadrivariate(
    dict(longitude=mx.flatten(),
         latitude=my.flatten(),
         time=mz.flatten(),
         level=mu.flatten())).reshape(mx.shape)

Bicubic on 4D grid
----------------------

The grid used organizes the latitudes in descending order. We ask our
constructor to flip this axis in order to correctly evaluate the bicubic
interpolation from this 4D cube (only necessary to perform a bicubic
interpolation).

In [ ]:
interpolator = pyinterp.backends.xarray.Grid4D(ds.pressure,
                                               increasing_axes=True)

We interpolate our grid using a
[bicubic](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.backends.xarray.Grid4D.quadrivariate.html#pyinterp.backends.xarray.Grid4D.bicubic)
interpolation in space followed by a linear interpolation in the temporal axis:

In [ ]:
bicubic = interpolator.bicubic(dict(longitude=mx.flatten(),
                                    latitude=my.flatten(),
                                    time=mz.flatten(),
                                    level=mu.flatten()),
                               nx=2,
                               ny=2).reshape(mx.shape)

We transform our result cubes into a matrix.

In [ ]:
quadrivariate = quadrivariate.squeeze(axis=(2, 3))
bicubic = bicubic.squeeze(axis=(2, 3))
lons = mx[:, 0].squeeze()
lats = my[0, :].squeeze()

Let's visualize our results.

---
**Note**

The resolution of the example grid is very low (one pixel every one degree)
therefore the calculation window cannot find the required pixels at the edges to
calculate the interpolation correctly. See Chapter [Fill NaN
Values](https://pangeo-pyinterp.readthedocs.io/en/latest/auto_examples/ex_fill_undef.html)
to see how to address this issue.

---

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(5, 4))
ax1 = fig.add_subplot(
    211, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax1.pcolormesh(lons,
                     lats,
                     quadrivariate.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree())
ax1.coastlines()
ax1.set_title("Trilinear")

ax2 = fig.add_subplot(
    212, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax2.pcolormesh(lons,
                     lats,
                     bicubic.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree())
ax2.coastlines()
ax2.set_title("Spline & Linear in time")
fig.colorbar(pcm, ax=[ax1, ax2], shrink=0.8)
fig.show()